In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import RobustScaler
import  gc
import matplotlib.pyplot as plt


In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


#Data

In [ ]:
train_data = pd.read_csv('/content/drive/My Drive/UNSW_NB15_training-set.csv')
test_data =pd.read_csv('/content/drive/My Drive/UNSW_NB15_testing-set.csv')

In [ ]:
train_data.isna().sum().sum()

0

In [ ]:
test_data.isna().sum().sum()

0

In [ ]:
train_data.shape

(82332, 45)

In [ ]:
test_data.shape

(175341, 45)

In [ ]:
train_data.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.000011,udp,-,INT,2,0,496,0,90909.0902,...,1,2,0,0,0,1,2,0,Normal,0
1,2,0.000008,udp,-,INT,2,0,1762,0,125000.0003,...,1,2,0,0,0,1,2,0,Normal,0
2,3,0.000005,udp,-,INT,2,0,1068,0,200000.0051,...,1,3,0,0,0,1,3,0,Normal,0
3,4,0.000006,udp,-,INT,2,0,900,0,166666.6608,...,1,3,0,0,0,2,3,0,Normal,0
4,5,0.000010,udp,-,INT,2,0,2126,0,100000.0025,...,1,3,0,0,0,2,3,0,Normal,0


In [ ]:
test_data.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.121478,tcp,-,FIN,6,4,258,172,74.087490,...,1,1,0,0,0,1,1,0,Normal,0
1,2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,...,1,2,0,0,0,1,6,0,Normal,0
2,3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,...,1,3,0,0,0,2,6,0,Normal,0
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,...,1,3,1,1,0,2,1,0,Normal,0
4,5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,...,1,40,0,0,0,2,39,0,Normal,0


#Data Preprocessing

In [ ]:
train_data.drop(['is_ftp_login','ct_ftp_cmd','is_sm_ips_ports'], inplace=True, axis=1)
test_data.drop(['is_ftp_login','ct_ftp_cmd','is_sm_ips_ports'], inplace=True, axis=1)

In [ ]:
#Selecting Categorical Features
categorical_cols = [col for col in train_data.columns if train_data[col].dtypes == 'object']
categorical_cols

['proto', 'service', 'state', 'attack_cat']

In [ ]:
#Handling categorical features using ordinal encoder
encoder = OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1)
encoder.fit(train_data[categorical_cols])
def ohe_new_features(df, features_names, encoder):
  new_feats = encoder.transform(df[features_names])
  new_cols = pd.DataFrame(new_feats, columns=features_names,dtype=int)
  drop_df = df.drop(features_names, axis=1)
  new_df = pd.concat([drop_df, new_cols], axis=1)
  
  gc.collect()
  return new_df

In [ ]:
train_data=ohe_new_features(train_data, categorical_cols, encoder)
test_data=ohe_new_features(test_data, categorical_cols, encoder)

In [ ]:
#scaling the data using Robust scaler
scaler = RobustScaler()
scaler.fit(train_data)
def scaling_data(df,scaler,cols):
  scaled_df = scaler.transform(df)
  df = pd.DataFrame(scaled_df,columns = cols)
  gc.collect()
  return  df

In [ ]:
train=scaling_data(train_data,scaler,train_data.columns.tolist())
test=scaling_data(test_data,scaler,train_data.columns.tolist())
train_data=pd.DataFrame(train,columns=train_data.columns.tolist())
test_data=pd.DataFrame(test,columns=test_data.columns.tolist())

In [ ]:
print(train.shape)
print(test.shape)

(82332, 42)
(175341, 42)


In [ ]:
y_train=train['attack_cat']
y_test=test['attack_cat']

In [ ]:
train.drop('attack_cat', inplace=True, axis=1)
test.drop('attack_cat', inplace=True, axis=1)

In [ ]:
X_train=(train.to_numpy())
X_test=(test.to_numpy())
y_train=y_train.to_numpy()
y_test=y_test.to_numpy()

In [ ]:
#!pip install fast_ml

In [ ]:
#!pip install interpret

#CGANs

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
from torchvision import datasets, transforms


In [ ]:
batch_size=512
y_train = torch.tensor(y_train, dtype=torch.int64)
y_test = torch.tensor(y_test, dtype=torch.int64)
trainloader = DataLoader(TensorDataset(torch.Tensor(X_train), y_train), batch_size=batch_size, shuffle=True)
testloader = DataLoader(TensorDataset(torch.Tensor(X_test), y_test), batch_size=batch_size, shuffle=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
class Generator(nn.Module):  
  def __init__(self):
    super(Generator, self).__init__()
    self.generator = nn.Sequential(
      nn.Linear(100, 128),
      nn.Linear(128, 256),
      nn.ReLU(True),
      nn.Linear(256, 512),
      nn.ReLU(True),
      nn.Linear(512, 39)
    )
    self.generator.apply(self.__init_weights)

  def forward(self, z, y):
    y = F.one_hot(y, num_classes=10)
    z = torch.cat((z, y), 1)
    x = self.generator(z)
    return x
  
  def __init_weights(self,m):
    #Init the weights (optional)
    if type(m) == nn.Linear:
      torch.nn.init.xavier_uniform_(m.weight)
      m.bias.data.fill_(0.01)
 

In [ ]:
class Discriminator(nn.Module):
  def __init__(self, h_dim=128):
    super(Discriminator, self).__init__()
    self.discriminator = nn.Sequential(
      nn.Linear(49, 512),
      nn.Linear(512, 256),
      nn.LeakyReLU(inplace=True),
      nn.Linear(256, 128),
      nn.LeakyReLU(inplace=True),
      nn.Linear(128, 1),
      nn.Sigmoid()
    )
    self.discriminator.apply(self.__init_weights)
  def forward(self, x, y):
    y = F.one_hot(y, num_classes=10)
    x = torch.cat((x, y), 1)
    x = self.discriminator(x)
    return x
  def __init_weights(self,m):
    if type(m) == nn.Linear:
      torch.nn.init.xavier_uniform_(m.weight)
      m.bias.data.fill_(0.01)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def Discriminator_Loss(o):
  loss=nn.BCELoss()
  x,y=o.squeeze(),torch.FloatTensor(o.size(0)).uniform_(0.9, 1).to(device)
  return loss(x,y)

In [ ]:
def Genenerator_Loss(o):
  loss=nn.BCELoss()
  x,y=o.squeeze(),torch.FloatTensor(o.size(0)).uniform_(0, 0.1).to(device)
  return loss(x,y)

In [ ]:
generator,discriminator = Generator().to(device),Discriminator().to(device)
generator_opt = torch.optim.SGD(generator.parameters(), lr=5e-4)
discriminator_opt = torch.optim.SGD(discriminator.parameters(), lr=5e-4)

In [ ]:
torch.cuda.is_available()

False

In [ ]:
num_epochs=40
for epoch in range(num_epochs):
    for batch_i, (X, y) in enumerate(trainloader):
        batch_size = X.size(0)
        z = np.random.uniform(-1, 1, size=(batch_size, 90))
        z = torch.from_numpy(z).float().to(device)
        generator_opt.zero_grad()
        X, y = X.to(device), y.to(device)
        dR= discriminator(X, y)
        DLoss = Discriminator_Loss(dR)
        not_G = generator(z, y)
        not_D = discriminator(not_G, y)
        Discriminator_Loss = Discriminator_Loss + Genenerator_Loss(not_D)
        Discriminator_Loss.backward()
        generator_opt.step()
        generator_opt.zero_grad()
        not_G = generator(z, y)
        Genenerator_Loss = Discriminator_Loss(discriminator(not_G, y))
        Genenerator_Loss.backward()
        generator_opt.step()
    print('epoch [{}/{}], Discriminator_Loss:{:.4f}, Genenerator_Loss:{:.4f}'.format(epoch + 1, num_epochs, Discriminator_Loss.item() , Genenerator_Loss.item()))

epoch [1/30], Discriminator_Loss: 0.9565 , Genenerator_Loss: 1.6853
epoch [2/30], Discriminator_Loss: 0.8503 , Genenerator_Loss: 3.4615
epoch [3/30], Discriminator_Loss: 0.8733 , Genenerator_Loss: 1.6990
epoch [4/30], Discriminator_Loss: 0.7875 , Genenerator_Loss: 1.7513
epoch [5/30], Discriminator_Loss: 0.8364 , Genenerator_Loss: 1.6031
epoch [6/30], Discriminator_Loss: 0.9812 , Genenerator_Loss: 2.9553
epoch [7/30], Discriminator_Loss: 3.0306 , Genenerator_Loss: 1.6723
epoch [8/30], Discriminator_Loss: 2.8908 , Genenerator_Loss: 1.7566
epoch [9/30], Discriminator_Loss: 0.8980 , Genenerator_Loss: 1.6771
epoch [10/30], Discriminator_Loss: 0.8927 , Genenerator_Loss: 1.5247
epoch [11/30], Discriminator_Loss: 0.8534 , Genenerator_Loss: 1.5631
epoch [12/30], Discriminator_Loss: 0.9190 , Genenerator_Loss: 1.6594
epoch [13/30], Discriminator_Loss: 0.8707 , Genenerator_Loss: 3.7768
epoch [14/30], Discriminator_Loss: 0.8958 , Genenerator_Loss: 1.5155
epoch [15/30], Discriminator_Loss: 0.7684 ,

#Classification using Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from interpret.glassbox import ExplainableBoostingClassifier
from sklearn.metrics import classification_report


In [ ]:
clf=RandomForestClassifier()

In [ ]:
clf.fit(X_train,y_train)

RandomForestClassifier()

In [ ]:
y_pred=clf.predict(X_test)

In [ ]:
print(classification_report(y_pred,y_test))


             precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.05      0.97      0.10        92
           2       0.56      0.34      0.43     19938
           3       0.65      0.73      0.69     29919
           4       0.86      0.90      0.88     17437
           5       0.98      0.94      0.96     41827
           6       1.00      1.00      1.00     56006
           7       0.74      0.92      0.82      8406
           8       0.65      0.44      0.52      1685
           9       0.17      0.71      0.27        31

    accuracy                           0.85    175341
   macro avg       0.57      0.70      0.57    175341
weighted avg       0.86      0.85      0.85    175341




#classification using Explainable Boosting Classifier

In [ ]:
clf = ExplainableBoostingClassifier()


In [ ]:
clf.fit(X_train,y_train)

In [ ]:
y_pred=clf.predict(X_test)


In [ ]:
print(classification_report(y_pred,y_test))


              precision    recall  f1-score   support

           0       0.96      0.75      0.85     37000
           1       0.01      0.13      0.04       583
           2       0.01      0.04      0.03       677
           3       0.28      0.57      0.38      6062
           4       0.33      0.70      0.46       378
           5       0.90      0.79      0.86      3496
           6       0.63      0.77      0.69     11132
           7       0.61      0.10      0.19      4089
           8       0.49      0.21      0.29        44
           9       1.00      0.97      0.98     18871

    accuracy                           0.86     82332
   macro avg       0.53      0.51      0.48     82332
weighted avg       0.84      0.76      0.78     82332




In [ ]:
class SN(nn.Module):
    
  def __init__(self):
    super(SN, self).__init__()
    self.clf = nn.Sequential(
      nn.Linear(50, 40),
      nn.ReLU(True),
      nn.Linear(40, 20)
    )
    
    self.clf.apply(self.__init_weights)

  def forward(self, x):
    output = self.clf(x)
    return output
  
  def __init_weights(self,m):
    #Init the weights (optional)
    if type(m) == nn.Linear:
      torch.nn.init.xavier_uniform_(m.weight)
      m.bias.data.fill_(0.01)

In [ ]:
def train_SN(model, data):
    optimizer = torch.optim.SGD(model.parameters(), lr=.001)
    criterion = nn.CrossEntropyLoss()
    model.train()
    for epoch in range(10):
        for X, y in data:
            X = X.to(device)
            y = y.to(device)
            
            optimizer.zero_grad()
            output = model(X)
            loss = criterion(output, y)
            
            loss.backward()
            optimizer.step()
        # log
        print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, loss.item()))
    return model


epoch [1/10], loss:0.7594
epoch [2/10], loss:0.7417
epoch [3/10], loss:0.5048
epoch [4/10], loss:0.6801
epoch [5/10], loss:0.4803
epoch [6/10], loss:0.5912
epoch [7/10], loss:0.6755
epoch [8/10], loss:0.3603
epoch [9/10], loss:0.6777
epoch [10/10], loss:0.3352



In [ ]:
clf= ExplainableBoostingClassifier()

In [ ]:
SN_clf = SN().to(device)

In [ ]:
train_SN(clf,trainloader)
clf.eval()

In [ ]:
output=clf(X_test.to(device))
y_pred = torch.max(output.data, 1)[1].cpu().detach().numpy()

In [ ]:
print(classification_report(y_pred,y_test))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.01      0.01      0.00         0
           2       0.08      0.23      0.14      3366
           3       0.55      0.64      0.54     25192
           4       0.12      0.54      0.22      4818
           5       0.95      0.67      0.86     51049
           6       0.96      0.65      0.74     90769
           7       0.02      0.06      0.00       137
           8       0.00      0.00      0.00         2
           9       0.01      0.01      0.00         0

    accuracy                           0.66    175341
   macro avg       0.25      0.28      0.24    175341
weighted avg       0.87      0.65      0.73    175341

